In [1]:
import numpy as np

with open('data/15.txt', 'r') as f:
    data = f.read()

    
def get_start_pos(mapa):
    start_pos = np.where(mapa=='@')
    return start_pos[0][0], start_pos[1][0]

def reload_map():
    mapa, actions = data.split('\n\n')
    actions = actions.replace('\n', '')
    mapa = np.array([list(x) for x in mapa.split('\n')])
    start_pos = get_start_pos(mapa)
    return mapa, actions, start_pos


mapa, actions, start_pos = reload_map()

In [2]:
action_map = {
    '>': (0, 1),
    '<': (0, -1),
    '^': (-1, 0),
    'v': (1, 0)
}

In [3]:
def step(cur_pos, action):
    action_dir = action_map[action]
    next_pos = cur_pos[0] + action_dir[0], cur_pos[1] + action_dir[1]
    if mapa[next_pos] == '#':
        return cur_pos
    if mapa[next_pos] == '.':
        mapa[cur_pos] = '.'
        mapa[next_pos] = '@'
        return next_pos
    else:
        last_pos = next_pos
        while mapa[last_pos] == 'O':
            last_pos = last_pos[0] + action_dir[0], last_pos[1] + action_dir[1]
        if mapa[last_pos] == '#':
            return cur_pos
        if action == 'v':
            mapa[
                next_pos[0] + 1: last_pos[0] + 1, next_pos[1]] = 'O'
        elif action == '^':
            mapa[
                last_pos[0]: next_pos[0], next_pos[1]] = 'O'
        elif action == '>':
            mapa[
                next_pos[0], next_pos[1]+1: last_pos[1]+1] = 'O'
        elif action == '<':
            mapa[
                next_pos[0], last_pos[1]: next_pos[1]] = 'O'
        mapa[next_pos] = '@'
        mapa[cur_pos] = '.'
        return next_pos

In [4]:
pos = start_pos
for a in actions:
    pos = step(pos, a)

In [5]:
def count_score(mapa, v='O'):
    res = 0
    ys, xs = np.where(mapa==v)
    ys = [int(x) for x in ys]
    xs = [int(x) for x in xs]
    for y, x in zip(ys, xs):
        res += 100 * y + x
    return res

In [6]:
count_score(mapa)

1478649

In [7]:
def reload_b_map():
    mapa, actions, start_pos = reload_map()
    mapa_b = np.full((mapa.shape[0], mapa.shape[1] * 2), '#')
    mapa_b[1:-1, 2:-2] = '.'
    ys, xs = np.where(mapa=='O')
    ys = [int(x) for x in ys]
    xs = [int(x) for x in xs]
    for y, x in zip(ys, xs):
        mapa_b[y, x*2: x*2+2] = ['[', ']']
    ys, xs = np.where(mapa=='#')
    ys = [int(x) for x in ys]
    xs = [int(x) for x in xs]
    for y, x in zip(ys, xs):
        mapa_b[y, x*2: x*2+2] = ['#', '#']
    mapa_b[start_pos[0], start_pos[1]*2] = '@'
    return mapa_b, (start_pos[0], start_pos[1]*2), actions

In [8]:
def print_b_map():
    for l in mapa_b:
        print(''.join(l))

In [9]:
def check_all_above(pos):
    new_values = set([pos])
    pos_up = pos[0] - 1, pos[1]
    if mapa_b[pos_up] == '.':
        return True, new_values
    elif mapa_b[pos_up] == '#':
        return False, {}
    
    possible_1, new_1 = check_all_above(pos_up)
    if mapa_b[pos_up] == '[':
        possible_2, new_2 = check_all_above((pos[0] - 1, pos[1] + 1))
    elif mapa_b[pos_up] == ']':
        possible_2, new_2 = check_all_above((pos[0] - 1, pos[1] - 1))
    if possible_1 and possible_2:
        return True, new_values | new_1 | new_2
    else:
        return False, {}

def fill_all_above(values):
    new_poses = {}
    for new_pos in values:
        if (new_pos[0]+1, new_pos[1]) not in values:
            new_poses[new_pos] = '.'
        else:
            new_poses[new_pos] = mapa_b[new_pos[0]+1, new_pos[1]]
        new_poses[new_pos[0]-1, new_pos[1]] = mapa_b[new_pos]
    for pose, value in new_poses.items():
        mapa_b[pose] = value

In [10]:
def check_all_below(pos):
    new_values = set([pos])
    pos_up = pos[0] + 1, pos[1]
    if mapa_b[pos_up] == '.':
        return True, new_values
    elif mapa_b[pos_up] == '#':
        return False, {}
    
    possible_1, new_1 = check_all_below(pos_up)
    if mapa_b[pos_up] == '[':
        possible_2, new_2 = check_all_below((pos[0] + 1, pos[1] + 1))
    elif mapa_b[pos_up] == ']':
        possible_2, new_2 = check_all_below((pos[0] + 1, pos[1] - 1))
    if possible_1 and possible_2:
        return True, new_values | new_1 | new_2
    else:
        return False, {}

def fill_all_below(values):
    new_poses = {}
    for new_pos in values:
        if (new_pos[0]-1, new_pos[1]) not in values:
            new_poses[new_pos] = '.'
        else:
            new_poses[new_pos] = mapa_b[new_pos[0]-1, new_pos[1]]
        new_poses[new_pos[0]+1, new_pos[1]] = mapa_b[new_pos]
    for pose, value in new_poses.items():
        mapa_b[pose] = value

In [11]:
def step_2(cur_pos, action):
    possible = True
    action_dir = action_map[action]
    next_pos = cur_pos[0] + action_dir[0], cur_pos[1] + action_dir[1]
    if mapa_b[next_pos] == '#':
        return cur_pos
    if mapa_b[next_pos] == '.':
        mapa_b[cur_pos] = '.'
        mapa_b[next_pos] = '@'
        return next_pos
    if action in ['>', '<']: # move left, right
        last_pos = next_pos
        while mapa_b[last_pos] in list('[]'):
            last_pos = last_pos[0] + action_dir[0], last_pos[1] + action_dir[1]
        if mapa_b[last_pos] == '#':
            return cur_pos
        if action == '>':
            mapa_b[
                next_pos[0], next_pos[1]+1: last_pos[1]+1] = mapa_b[next_pos[0], next_pos[1]: last_pos[1]]
        elif action == '<':
            mapa_b[
                next_pos[0], last_pos[1]: next_pos[1]] = mapa_b[next_pos[0], last_pos[1] +1: next_pos[1] + 1]
        mapa_b[next_pos] = '@'
        mapa_b[cur_pos] = '.'
            
    elif action == '^':
        possible, new_values = check_all_above(cur_pos)
        if possible:
            fill_all_above(new_values)
    else:
        possible, new_values = check_all_below(cur_pos)
        if possible:
            fill_all_below(new_values)
    return next_pos if possible else cur_pos

In [12]:
mapa_b, start_pos, actions = reload_b_map()

In [13]:
pos = start_pos
for a in actions:
    pos = step_2(pos, a)

In [14]:
count_score(mapa_b, '[')

1495455